In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt

import string
from nltk.util import ngrams
from collections import Counter
from sklearn.preprocessing import PolynomialFeatures


from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingRegressor, ExtraTreesRegressor, RandomForestRegressor
from sklearn.linear_model import LogisticRegression


In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt

import string
from nltk.util import ngrams
from collections import Counter
from sklearn.preprocessing import PolynomialFeatures


from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingRegressor, ExtraTreesRegressor, RandomForestRegressor
from sklearn.linear_model import LogisticRegression

In [5]:
import re

In [6]:
sample_submission = pd.read_csv('/content/sample_submission.csv')

In [7]:
sample_submission

,sell_id,price
0,1100575026,0
1,1100549428,0
2,1100658222,0
3,1100937408,0
4,1101037972,0
...,...,...
34681,1101369263,0
34682,1101369581,0
34683,1101364889,0
34684,1101362518,0


In [8]:
RANDOM_SEED = 42
VAL_SIZE   = 0.2

In [9]:

data = pd.read_csv('/content/data2201.csv') 

In [10]:
data =data.drop_duplicates()

In [11]:
data

,Unnamed: 0,bodyType,brand,color,engineDisplacement,enginePower,fuelType,mileage,modelDate,model_name,numberOfDoors,productionDate,vehicleTransmission,vendor,Владельцы,ПТС,Привод,Руль,leather,carter,ABS,airbags,immob,central_locking,on_board_computer,cruise_control,climat_control,multi_rudder,power_steering,light_and_rain_sensors,сarbon_body_kits,rear_diffuser_rkp,door_closers,rear_view_camera,amg,bi_xenon_headlights,from_salon,alloy_wheels,parking_sensors,dents,roof_with_panoramic_view,bT_length,Nalog,years_old,age_intervals,sample
0,0,9,8,14,1.2,105.0,0,74000,2013,713,5,2014,3,2,2,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,2625.0,7,1,0
1,1,9,8,16,1.6,110.0,0,60563,2017,713,5,2017,2,2,0,1,1,0,0,0,1,0,1,1,1,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,7,2750.0,4,1,0
2,2,9,8,13,1.8,152.0,0,88000,2013,957,5,2014,3,2,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,6840.0,7,1,0
3,3,9,8,7,1.6,110.0,0,95000,2013,713,5,2014,0,2,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,2750.0,7,1,0
4,4,9,8,1,1.8,152.0,0,58536,2008,713,5,2012,0,2,0,1,1,0,0,0,1,0,1,1,1,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,7,6840.0,9,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231346,231346,16,11,14,2.4,170.0,0,120000,2000,859,4,2003,0,6,2,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,7650.0,18,3,1
231347,231347,20,11,16,2.0,160.0,0,230000,1995,1026,5,1998,2,6,2,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,15,7200.0,23,4,1
231348,231348,16,11,12,2.3,131.0,0,100000,1983,70,4,1985,2,6,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,4585.0,36,5,1
231349,231349,16,11,16,1.6,109.0,0,245000,1999,856,4,2001,2,6,2,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,2725.0,20,3,1


In [12]:
columns_drop = ['Unnamed: 0']
data = data.drop(columns_drop, axis=1)

In [13]:
a = pd.read_csv('/content/yy2201.csv')

In [14]:

a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196665 entries, 0 to 196664
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  196665 non-null  int64  
 1   price       196665 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 3.0 MB


In [15]:
y = a['price']

In [16]:
y

0         200000.0
1          60000.0
2          99000.0
3          65000.0
4         100000.0
            ...   
196660    280000.0
196661    230000.0
196662    150000.0
196663    170000.0
196664    750000.0
Name: price, Length: 196665, dtype: float64

In [17]:
# columns_drop = ['Unnamed: 0']
# data = data.drop(columns_drop, axis=1)

In [18]:
X = data.query('sample == 1').drop(['sample'], axis=1)
X_sub = data.query('sample == 0').drop(['sample'], axis=1)

In [19]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, random_state=RANDOM_SEED)

In [21]:
np.log(y_train)

171013    13.997832
120091    13.369223
122665    13.910821
114495    13.963931
109074    12.611538
            ...    
119879    13.763164
103694    12.254863
131932    16.077065
146867    13.440090
121958    12.206073
Name: price, Length: 157332, dtype: float64

In [29]:
data['rinat']

KeyError: ignored

# **Boosting Algorithms**



 **Extreme Gradient Boosting Algorithm.**

> Result in Kaggle submissions = 8.04237



In [34]:
import xgboost as xgb

In [35]:
xg_reg = xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.5,
                          learning_rate= 0.01 ,max_depth=24, alpha=1,
                          n_estimators=15000,tree_method='gpu_hist')


In [36]:
xg_reg.fit(X_train, np.log(y_train))

XGBRegressor(alpha=1, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.5, gamma=0,
             importance_type='gain', learning_rate=0.01, max_delta_step=0,
             max_depth=24, min_child_weight=1, missing=None, n_estimators=15000,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, tree_method='gpu_hist',
             verbosity=1)

In [37]:
predict_test = np.exp(xg_reg.predict(X_test)) 
predict_submission = np.exp(xg_reg.predict(X_sub))


In [38]:
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

Точность модели по метрике MAPE: 10.30%


In [39]:
sample_submission['price'] = predict_submission
sample_submission.to_csv('submission_xgboost22fin.csv', index=False)


In [ ]:
import pickle

**GradientBoostingRegressor**

> Result in Kaggle submissions = 10.37510

In [25]:
gb = GradientBoostingRegressor(min_samples_split=2, learning_rate=0.01, max_depth=24, n_estimators=1000)
gb.fit(X_train, np.log(y_train))

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.01, loss='ls',
                          max_depth=24, max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=1000,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [26]:
predict_test = np.exp(gb.predict(X_test))
predict_submission = np.exp(gb.predict(X_sub))

In [ ]:
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

In [28]:
sample_submission['price'] = predict_submission
sample_submission.to_csv('submission_2201gb.csv', index=False)


# Bagging

# StackingRegressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor


In [ ]:
import xgboost as xgb

In [ ]:
estimators = [
    ('xg_reg', xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.5,
                          learning_rate= 0.01 ,max_depth=26, alpha=1,
                          n_estimators=5000,tree_method='gpu_hist')),
    ('gb', GradientBoostingRegressor(min_samples_split=2, learning_rate=0.01, max_depth=15,
                                     n_estimators=5000))
]

In [ ]:
reg5 = StackingRegressor(estimators=estimators)
reg5.fit(X_train, np.log(y_train))


In [ ]:
predict_test = np.exp(reg5.predict(X_test))
predict_submission = np.exp(reg5.predict(X_sub))


In [ ]:
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

In [ ]:
sample_submission['price'] = predict_submission
sample_submission.to_csv('submission_2316st.csv', index=False)
            